In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import os

# Create a smaller example graph with 6 nodes
def create_small_graph():
    G = nx.Graph()
    edges = [
        (1, 2), (1, 3), (2, 3),  # Clique {1, 2, 3}
        (4, 5), (4, 6), (5, 6),  # Clique {4, 5, 6}
        (1, 4)                    # Connect cliques
    ]
    G.add_edges_from(edges)
    return G

# Bron-Kerbosch algorithm with step-by-step visualization
retained_cliques = []  # To track cliques as they form

# Global step counter to ensure unique visualization for PowerPoint
step_counter = 0

# Create folder for output images
output_folder = "bron_kerbosch_steps"
os.makedirs(output_folder, exist_ok=True)

def bron_kerbosch_with_visualization(graph, r, p, x, cliques, depth=0):
    global step_counter
    indent = "  " * depth  # Indentation for better readability in logging
    step_description = f"Step {step_counter}: R={r}, P={p}, X={x}"
    print(f"{indent}{step_description}")

    # Visualize current step
    visualize_step(graph, r, p, x, cliques, step_counter, step_description)
    step_counter += 1

    if not p and not x:
        cliques.append(r)
        retained_cliques.append(set(r))  # Retain cliques as they are found
        print(f"{indent}--> Found maximal clique: {r}")
        return

    for v in list(p):  # Iterate over a copy of p to modify it safely
        neighbors = set(graph.neighbors(v))
        print(f"{indent}Exploring node {v}...")
        bron_kerbosch_with_visualization(graph, r | {v}, p & neighbors, x & neighbors, cliques, depth + 1)
        p.remove(v)
        x.add(v)
        print(f"{indent}Backtracking: removed {v} from P, added to X")

# Visualize the state of the algorithm at each step and save as image
def visualize_step(graph, r, p, x, cliques, step, step_description):
    pos = nx.spring_layout(graph, seed=42)
    plt.figure(figsize=(6, 4))
    nx.draw(graph, pos, with_labels=True, node_size=400, font_size=9, font_color="white", edge_color="gray")

    # Highlight nodes in R, P, and X
    nx.draw_networkx_nodes(graph, pos, nodelist=list(r), node_color="red", label="R (Current Clique)")
    nx.draw_networkx_nodes(graph, pos, nodelist=list(p), node_color="green", label="P (Potential Nodes)")
    nx.draw_networkx_nodes(graph, pos, nodelist=list(x), node_color="blue", label="X (Excluded Nodes)")

    # Draw retained cliques with different colors
    colors = plt.cm.tab10.colors
    for idx, clique in enumerate(retained_cliques):
        nx.draw_networkx_nodes(graph, pos, nodelist=list(clique), node_color=[colors[idx % len(colors)]], alpha=0.5, label=f"Retained Clique {idx+1}")

    plt.title(step_description)
    plt.legend(loc="upper left", fontsize=10)

    # Save the image
    file_path = os.path.join(output_folder, f"step_{step}.jpg")
    plt.savefig(file_path, format="jpg")
    plt.close()

# Main Execution
small_graph = create_small_graph()
found_cliques = []
print("Starting Bron-Kerbosch algorithm with visualization...")
bron_kerbosch_with_visualization(small_graph, set(), set(small_graph.nodes()), set(), found_cliques)

print("\nMaximal cliques found:", found_cliques)


Starting Bron-Kerbosch algorithm with visualization...
Step 0: R=set(), P={1, 2, 3, 4, 5, 6}, X=set()
Exploring node 1...
  Step 1: R={1}, P={2, 3, 4}, X=set()
  Exploring node 2...
    Step 2: R={1, 2}, P={3}, X=set()
    Exploring node 3...
      Step 3: R={1, 2, 3}, P=set(), X=set()
      --> Found maximal clique: {1, 2, 3}
    Backtracking: removed 3 from P, added to X
  Backtracking: removed 2 from P, added to X
  Exploring node 3...
    Step 4: R={1, 3}, P=set(), X={2}
  Backtracking: removed 3 from P, added to X
  Exploring node 4...
    Step 5: R={1, 4}, P=set(), X=set()
    --> Found maximal clique: {1, 4}
  Backtracking: removed 4 from P, added to X
Backtracking: removed 1 from P, added to X
Exploring node 2...
  Step 6: R={2}, P={3}, X={1}
  Exploring node 3...
    Step 7: R={2, 3}, P=set(), X={1}
  Backtracking: removed 3 from P, added to X
Backtracking: removed 2 from P, added to X
Exploring node 3...
  Step 8: R={3}, P=set(), X={1, 2}
Backtracking: removed 3 from P, added